In [1]:
import json

with open("semeval_testV4.json", "r") as f:
    data = json.load(f)
    
import spacy

# data but only keep examples that are labeled as causal
causal_data = [ d for d in data if d["relation_type"] == 0][:]





In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
nouns_and_verbs = []
print(causal_data[2]["sent"])

#get token dependencies
for t in docs[2]:
    word = {}
    if t.orth_ == "from":
        print("from", t.pos_, t.dep_)
    if t.pos_ == "NOUN" or t.pos_ == "PROPN" or t.pos_ == "VERB":
        word["pos"] = t.pos_
        word["text"] = str(t.orth_)
        word["index"] = t.i
        word["dep"] = t.dep_
        nouns_and_verbs.append(word)
print(nouns_and_verbs)

Of the hundreds of strains of avian influenza A viruses, only four have caused human infections: H5N1, H7N3, H7N7, and H9N2.


NameError: name 'docs' is not defined

In [5]:
def generate_sent_info(sent):
    doc = nlp(sent)
    nouns_and_verbs = []
    for t in doc:
        word = {}
        if t.pos_ == "NOUN" or t.pos_ == "PROPN" or t.pos_ == "VERB":
            word["pos"] = t.pos_
            word["text"] = str(t.orth_)
            word["index"] = t.i
            word["dep"] = t.dep_
            nouns_and_verbs.append(word)
    return nouns_and_verbs

In [6]:
def spacy_doc_data(data):
    word_data = []
    for d in data:
        n_and_v = generate_sent_info(d["sent"])
        word_data.append(n_and_v)
    return word_data

In [7]:
# remove nouns from consideration that are not separated by a verb
def filter_contiguous_nouns(n_v_list, pairings):
    filtered = []
    for pairing in pairings:
        item_1_index = pairing[0]["index"]
        item_2_index = pairing[1]["index"]
        for word in n_v_list:
            if word["pos"] == "VERB":
                if word["index"] > item_1_index and word["index"] < item_2_index:
                    filtered.append(pairing)
    return filtered
    

In [8]:
#
def filter_dep_types(pairings):
    filtered = []
    for pairing in pairings:
        if pairing[0]["dep"] in ["nsubj", "nsubjpass", "dobj", "pobj", "iobj", "nsubj","advcl", "meta", "xcomp", "conj"]:
            if pairing[1]["dep"] in ["nsubj", "nsubjpass", "dobj", "pobj", "iobj", "nsubj","advcl", "meta", "xcomp", "conj"]:
                filtered.append(pairing)
    return filtered
    

In [9]:
# get all possible cause effect pairs in a sentence using nouns and verbs list
def get_possible_relations(n_v_list):
    pairings = []
    for i, word in enumerate(n_v_list):
        for j in range(i + 1, len(n_v_list)):
            word2 = n_v_list[j]
            if word["pos"] != "VERB" and word2["pos"] != "VERB":
                pairings.append((word, word2))
    return pairings
                    
                    
        
    

In [10]:
def evaluate_containment(dataset, pairings_list):
    num_contained = 0
    for i, d in enumerate(dataset):
        e1 = d["e1_contents"]
        e2 = d["e2_contents"]
        pairings = pairings_list[i]
        temp = num_contained
        for pairing in pairings:
            if pairing[0]["text"] in e1 or pairing[0]["text"] in e2:
                if pairing[1]["text"] in e1 or pairing[1]["text"] in e2:
                    num_contained += 1
                    break
        if num_contained == temp:
            pass
            #print(d["sent"], pairing)
    return num_contained
                    
        

In [11]:
#print(len(nouns_and_verbs))

print(len(causal_data))
n_and_v_list = spacy_doc_data(causal_data)

pairings_list = []
for n_and_v in n_and_v_list:
    pairings_list.append(get_possible_relations(n_and_v))
    
overall_num_pairings = 0
for pairing in pairings_list:
    overall_num_pairings += len(pairing)
    
filtered_list = []
for i, pairing in enumerate(pairings_list):
    filtered_list.append(filter_dep_types(pairing))

num_contained = evaluate_containment(causal_data, filtered_list)
print("contained", num_contained, "as percent", num_contained / len(filtered_list))

reduced_num_pairings = 0
for filtered in filtered_list:
    reduced_num_pairings += len(filtered)
print(overall_num_pairings)
print(reduced_num_pairings)

"""
print(len(pairings))
filtered = filter_contiguous_nouns(nouns_and_verbs, pairings)
print(len(filtered))
filtered_dep = filter_dep_types(filtered)
print(len(filtered_dep))
print(filtered_dep)

print("contained", evaluate_containment(causal_data, pairings_list))
"""



    
    
    
    

323
contained 287 as percent 0.8885448916408669
6923
3794


'\nprint(len(pairings))\nfiltered = filter_contiguous_nouns(nouns_and_verbs, pairings)\nprint(len(filtered))\nfiltered_dep = filter_dep_types(filtered)\nprint(len(filtered_dep))\nprint(filtered_dep)\n\nprint("contained", evaluate_containment(causal_data, pairings_list))\n'

In [12]:
all_spacy_deps = ["ROOT",
"acl",
"acomp",
"advcl",
"advmod",
"agent",
"amod",
"appos",
"attr",
"aux",
"auxpass",
"case",
"cc",
"ccomp",
"compound",
"conj",
"csubj",
"csubjpass",
"dative",
"dep",
"det",
"dobj",
"expl",
"intj",
"mark",
"meta",
"neg",
"nmod",
"npadvmod",
"nsubj",
"nsubjpass",
"nummod",
"oprd",
"parataxis",
"pcomp",
"pobj",
"poss",
"preconj",
"predet",
"prep",
"prt",
"punct",
"quantmod",
"relcl",
"xcomp"]
for dep in all_spacy_deps:
    print(dep, "-", spacy.explain(dep))

ROOT - None
acl - clausal modifier of noun (adjectival clause)
acomp - adjectival complement
advcl - adverbial clause modifier
advmod - adverbial modifier
agent - agent
amod - adjectival modifier
appos - appositional modifier
attr - attribute
aux - auxiliary
auxpass - auxiliary (passive)
case - case marking
cc - coordinating conjunction
ccomp - clausal complement
compound - compound
conj - conjunct
csubj - clausal subject
csubjpass - clausal subject (passive)
dative - dative
dep - unclassified dependent
det - determiner
dobj - direct object
expl - expletive
intj - interjection
mark - marker
meta - meta modifier
neg - negation modifier
nmod - modifier of nominal
npadvmod - noun phrase as adverbial modifier
nsubj - nominal subject
nsubjpass - nominal subject (passive)
nummod - numeric modifier
oprd - object predicate
parataxis - parataxis
pcomp - complement of preposition
pobj - object of preposition
poss - possession modifier
preconj - pre-correlative conjunction
predet - None
prep - pr

In [12]:
def filter_dep_types_m(pairings, dep):
    filtered = []
    for pairing in pairings:
        if pairing[0]["dep"] in ["nsubj", "nsubjpass", "dobj", "pobj", "iobj", "ROOT", "acomp", "advcl", "meta", "xcomp","conj" dep]:
            if pairing[1]["dep"] in ["nsubj", "nsubjpass", "dobj", "pobj", "iobj", "nsubj","advcl", "meta", "xcomp","conj", dep]:
                filtered.append(pairing)
    return filtered

for dep in all_spacy_deps:
    print(dep, "-", spacy.explain(dep))
    
    n_and_v_list = spacy_doc_data(causal_data)

    pairings_list = []
    for n_and_v in n_and_v_list:
        pairings_list.append(get_possible_relations(n_and_v))

    overall_num_pairings = 0
    for pairing in pairings_list:
        overall_num_pairings += len(pairing)

    filtered_list = []
    for i, pairing in enumerate(pairings_list):
        filtered_list.append(filter_dep_types_m(pairing, dep))

    num_contained = evaluate_containment(causal_data, filtered_list)
    #print("contained", num_contained, "as percent", num_contained / len(filtered_list))

    reduced_num_pairings = 0
    for filtered in filtered_list:
        reduced_num_pairings += len(filtered)
    print(overall_num_pairings)
    if reduced_num_pairings == 8435:
        print("num contained", num_contained-768, "num pairings", 8435, "\npercentage", 0)
    else:
        print("num contained", num_contained-768, "num pairings", reduced_num_pairings-8435)
        print("percentage", ((num_contained-768)/(reduced_num_pairings-8435)))
    print("--------------------------------------")


SyntaxError: invalid syntax (<ipython-input-12-03006df408d6>, line 4)

In [15]:
total = 0
for filtered in filtered_list:
    total += len(filtered)
print(len(filtered_list))
print(total/len(filtered_list))
    
# get simple sentences with fewer pairs of nouns
simple = []
for i, sent_pairings in enumerate(filtered_list):
    if len(sent_pairings) > 15:
        pass
    else:
        simple.append((sent_pairings, i))
print(len(simple))
print(simple[0])


323
11.746130030959753
272
([({'pos': 'NOUN', 'text': 'influenza', 'index': 1, 'dep': 'nsubj'}, {'pos': 'NOUN', 'text': 'birds', 'index': 7, 'dep': 'pobj'}), ({'pos': 'NOUN', 'text': 'influenza', 'index': 1, 'dep': 'nsubj'}, {'pos': 'NOUN', 'text': 'strains', 'index': 12, 'dep': 'pobj'}), ({'pos': 'NOUN', 'text': 'influenza', 'index': 1, 'dep': 'nsubj'}, {'pos': 'NOUN', 'text': 'virus', 'index': 16, 'dep': 'pobj'}), ({'pos': 'NOUN', 'text': 'birds', 'index': 7, 'dep': 'pobj'}, {'pos': 'NOUN', 'text': 'strains', 'index': 12, 'dep': 'pobj'}), ({'pos': 'NOUN', 'text': 'birds', 'index': 7, 'dep': 'pobj'}, {'pos': 'NOUN', 'text': 'virus', 'index': 16, 'dep': 'pobj'}), ({'pos': 'NOUN', 'text': 'strains', 'index': 12, 'dep': 'pobj'}, {'pos': 'NOUN', 'text': 'virus', 'index': 16, 'dep': 'pobj'})], 0)


In [13]:
# Adds the e1 e2 tags back into a sentence
def insert_e1_e2_labels(sent, e1, e2):
    e1_start = sent.find(e1)
    e1_end = sent.find(" ", e1_start)
    if e1_start == -1:
        print("error unable to find", e1, "in", sent)
    e1_desig_s = "<e1>"
    e1_desig_e = "</e1>"
    sent = sent[:e1_start] + e1_desig_s + sent[e1_start:e1_end] + e1_desig_e + sent[e1_end:]
    
    e2_start = sent.find(e2)
    e2_end = sent.find(" ", e2_start)
    if e2_start == -1:
        print("error unable to find", e2, "in", sent)
    e2_desig_s = "<e2>"
    e2_desig_e = "</e2>"
    sent = sent[:e2_start] + e2_desig_s + sent[e2_start:e2_end] + e2_desig_e + sent[e2_end:]
    return sent
    
# Adds the e1 e2 labels to all datapoints in dataset
def add_e1_e2_labels(dataset):
    for i, d in enumerate(dataset):
        d["sent"] = insert_e1_e2_labels(d["sent"], d["e1_contents"], d["e2_contents"])
    return dataset
# alternative to add_e1_e2_labels
def add_e1_e2_0001_0002_labels(dataset):
    for i, d in enumerate(dataset):
        d["sent"] = replace_e1_e2_labels_unused_roberta(d["sent"], d["e1_contents"], d["e2_contents"])
    return dataset

# alternative to using e1 e2 labels
def replace_e1_e2_labels_unused_roberta(sent, e1, e2):
    sent = sent.replace(e1, "0001")
    sent = sent.replace(e2, "0002")
    return sent


In [16]:
# create augmented data

def convert_pairings_to_data(sent, pairing):
    sent = replace_e1_e2_labels_unused_roberta(sent, pairing[0]["text"], pairing[1]["text"])
    return sent
    
#simple list item - tuple_num - access tuple - tuple item

def convert_all(data, simple):
    augmented = []
    for s in simple:
        pairings = s[0]
        for p in pairings:
            aug = {}
            aug["sent"] = convert_pairings_to_data(data[s[1]]["sent"], p)
            aug["e1_contents"] = p[0]["text"]
            aug["e2_contents"] = p[1]["text"]
            aug["relation_type"] = 1
            e1 = data[s[1]]["e1_contents"]
            e2 = data[s[1]]["e2_contents"]
            #these if statements remove 
            if p[0]["text"] in e1 or p[0]["text"] in e2:
                    if p[1]["text"] in e1 or p[1]["text"] in e2:
                        continue
            augmented.append(aug)
    return augmented
augmented = convert_all(causal_data, simple[:])
print(len(augmented))
print(augmented)

# add in positive examples
c_data = add_e1_e2_0001_0002_labels(causal_data)
augmented = augmented + c_data




1836
[{'sent': 'Avian 0001 is an infectious disease of 0002 caused by type A strains of the 0001 virus.', 'e1_contents': 'influenza', 'e2_contents': 'birds', 'relation_type': 1}, {'sent': 'Avian 0001 is an infectious disease of birds caused by type A 0002 of the 0001 virus.', 'e1_contents': 'influenza', 'e2_contents': 'strains', 'relation_type': 1}, {'sent': 'Avian influenza is an infectious disease of 0001 caused by type A 0002 of the influenza virus.', 'e1_contents': 'birds', 'e2_contents': 'strains', 'relation_type': 1}, {'sent': 'Avian influenza is an infectious disease of 0001 caused by type A strains of the influenza 0002.', 'e1_contents': 'birds', 'e2_contents': 'virus', 'relation_type': 1}, {'sent': 'Avian influenza is an infectious disease of birds caused by type A 0001 of the influenza 0002.', 'e1_contents': 'strains', 'e2_contents': 'virus', 'relation_type': 1}, {'sent': 'The same 0001 is achieved the traditional way, with a 0002 of workers like Keebler elves.', 'e1_contents

In [17]:
import random
print(augmented[-1])
random.shuffle(augmented)


with open("semeval_testV4_step2.json", "w") as f:
    json.dump(augmented, f, indent = 4)

{'sent': 'She was looking beautiful in the dim 0001 that emanated from my broken 0002.', 'relation_type': 0, 'is_active': 0, 'e1_contents': 'light', 'e2_contents': 'table-lamp'}


In [49]:
maxa = -1
index = -1
for i, d in enumerate(augmented):
    if len(d) > maxa:
        maxa = len(d)
        index = i

print(index)
print(augmented[36])

36
{'sent': 'In these cases, the 0001 from the 0002 is not forgotten over time, but rather accumulates.', 'relation_type': 0, 'is_active': 0, 'e1_contents': 'disappointment', 'e2_contents': 'purchase', 'id': 'original:5101'}
